<a href="https://colab.research.google.com/github/richardcmg7/AIND-NLP/blob/master/Week_1_Project_Building_a_Leaf_Classification_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### This project is from [Abubakar Abid's](https://www.linkedin.com/in/abid15/) course: *Building Computer Vision Applications* on CoRise. Learn more about the course [here](https://corise.com/course/computer-vision).

# Week 1 Project: Building a Leaf Classification App

Welcome to the first week's project for *Building Computer Vision Applications*!

In this week, we are going to get familiar with the key steps of machine learning, with a particular focus on image classification. Specifically, we will cover:

* finding computer vision datasets and pretrained models 📖
* fine-tuning an image classifier model on new data 👾
* building a computer vision app you can run on your phone or laptop 📷
* measuring the performance of a classification model on test data and the real world 📈

# Introduction

Beans are an important cereal food crop in many parts of the world. However, certain diseases can damage bean plants, causing food shortages. As a result, it is critical to monitor the leaves of bean plants frequently and accurately. 

This is a great example of task where an **image classification** model can help automate this work. The concepts you will learn in this project will be generally applicable to many kinds of machine learning tasks. 

Our end goal, will be to build a web-application that can take in an image of a bean leaf and predict whether it is health or diseased. A prototype of an app like this: 

![](https://i.ibb.co/6mcXB53/image.png)

# Step 0: Hardware Setup & Software Libraries

We will be utilizing GPUs to train our machine learning model, so we will need to make sure that our colab notebook is set up correct. Go to the menu bar and click on Runtime > Change runtime type > Hardware accelerator and **make sure it is set to GPU**. Your colab notebook may restart once you make the change.

We're going to be using some fantastic open-source Python libraries to load our dataset (`datasets`), train our model (`transformers`), evaluate our model (`evaluate`), and build a demo of our model (`gradio`). So let's go ahead and install all of these libraries. 

In [ ]:
!pip install datasets transformers evaluate gradio 

# Step 1: Loading a Dataset

In this project, we will be using the `datasets` library, which can load tens of thousands of datasets with a single line of code. It can also be used to apply preprocessing functions. Learn more about the datasets library here: https://huggingface.co/docs/datasets/tutorial

Most datasets are divided into different splits. For example, you'll often see a *training* data subset, which is used to build the model, a *validation* data subset, which is used to measure the performance of the model while it is training, and a *test* dataset which is used to measure the performance of the model at the very end of training, and is usually considered how well the model will perform in the real world (we'll come back to this).

Specifically, we will be using the `beans` dataset that is available for free from the Hugging Face Hub: https://huggingface.co/datasets/beans

* **Load the Beans Dataset**

In [ ]:
import datasets

dataset = # FILL HERE

* **Explore the dataset by running the cells below and answer the questions below**

In [ ]:
print(dataset)

In [ ]:
for i in range(10):
  display(dataset['train'][i]['image'])

* What information do we have for each sample? [ANSWER HERE]
* How many training samples do we have? Validation samples? Test samples? [ANSWER HERE]
* What are the labels in this dataset? [ANSWER HERE]
* Look at the first 10 training images, do you notice anything interesting about the images in the dataset? Are they as diverse/representative as you would expect or do they have limitations? [ANSWER HERE]

# Step 2: Loading a Pretrained Model

We will be using the `transformers` library, which can load tens of thousands of machine learning models with a few lines of code. It can also be used to fine-tune these models. Learn more about the `transformers` library here: https://huggingface.co/docs/transformers/index

Specifically, we will be using the `Vision Image Transformer` model that is available for anyone from the Hugging Face Hub: https://huggingface.co/google/vit-base-patch16-224. While the details of vision transformers are beyond the scope of this course, we will point out that they are a successor of the widely used convolutional neural network (CNN) architecture and tend to perform better than CNNs at the same tasks (image classificaiton, segmentation, etc.)

Let's start by seeing how the Vision Image Transformer model performs without any further fine-tuning.

* **Load the Vision Image Transformer Model and FeatureExtractor for Inference**

In [ ]:
import transformers
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = # FILL HERE
model.eval()
model.to(device);

We also need to load the **feature extractor** corresponding to the model, so that we can convert the input images into a feature vector that the model can take as input.

In [ ]:
feature_extractor = # FILL HERE

* **Use the Vision Image Transformer Model to Make a Prediction on the Training Images**

The documentation here may be helpful: https://huggingface.co/docs/transformers/model_doc/vit#transformers.ViTForImageClassification.forward.example

In [ ]:
# First we get the features corresponding to the first training image
encoding = feature_extractor(images=dataset['train'][0]['image'], return_tensors="pt")

# Then pass it through the model and get a prediction

######
# FILL HERE
######

prediction = # FILL HERE

print("Predicted class:", model.config.id2label[prediction])

* Try running the model on the first 10 samples in the dataset. What is the most common prediction? Why do you think that is? [ANSWER HERE]

# Step 3: Finetuning Your Model on the Dataset

Off the shelf, the Vision Image Transformer will not be usable for the task that we have in mind, since it was trained for "general" image classification, not for the specific categories that we would like to predict. As a result, we will need to "fine-tune" our model.

Learn more about fine-tuning models with the `transformers` library here: https://huggingface.co/docs/transformers/training

We will also need to decide which metric to use for our task. Since our task is a simple image classification task, the `accuracy` metric seems reasonable: https://huggingface.co/spaces/evaluate-metric/accuracy

* **Preprocess the Dataset**

To make things faster, we are going to preprocess the entire dataset so that we convert all of the images to feature vectors. This will allow us to speed up the training as we can pass the feature vectors directly. This code has already been written for you:

In [ ]:
import torch

def transform(example_batch):
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')
    inputs['labels'] = example_batch['labels']
    return inputs

prepared_ds = dataset.with_transform(transform)

* **Load the Accuracy Metric**

We now have to decide on a *metric* we will use to measure the performance for our machine learning model. A natural choice for image classification is *accuracy*, which measures the percentage of images that are predicted to have the correct label. 

Read about the `evaluate` library, which contains many common machine learning metrics here: https://github.com/huggingface/evaluate

And use the `evaluate.load()` to load the accuracy metric:

In [ ]:
import numpy as np
import evaluate

metric = # FILL HERE

def compute_metrics(sample):
    return metric.compute(
        predictions=np.argmax(sample.predictions, axis=1), 
        references=sample.label_ids)

* **Fine-Tune the Vision Image Transformer Model on the Entire Training Set**

Now, we will take all of the code that you have written and use it to fine-tune the ViT model on the beans dataset. Simply run the code below, and your model will fine-tune for 4 epochs. On a **GPU**, this should take less than 5 minutes.

In [ ]:
from transformers import TrainingArguments
from transformers import Trainer

training_args = TrainingArguments(
  output_dir="./vit-base-beans",
  per_device_train_batch_size=16,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

# Step 4: Reporting Model Metrics

* **Measure Accuracy on the Validation Dataset**

In [ ]:
eval_accuracy = # FILL HERE

* What is the loss and the accuracy on the training set [ANSWER HERE]
* Is there any sign of overfitting? [ANSWER HERE]


* **Measure Accuracy on the Test Dataset**


In [ ]:
test_accuracy = # FILL HERE

* What is your final test accuracy? [ANSWER HERE]


# Step 5: Building a Demo

A high-level metric like test accuracy doesn't give us a great idea on how the model will work when presented with new data from the real world. To understand this, we will build a web-based demo that can be used on our phones or computers through a web browser to test our model.

The `gradio` library lets you build web demos of machine learning models with just a few lines code. Learn more about Gradio here: https://gradio.app/getting_started/

Gradio lets you build machine learning demos simply by specifying (1) a prediction function, (2) the input type and (3) the output type of your model. We have already written the prediction function here:

In [ ]:
labels = dataset['train'].features['labels'].names

def classify(im):
  features = feature_extractor(im, return_tensors='pt').to(device)
  logits = model(features["pixel_values"])[-1]
  probability = torch.nn.functional.softmax(logits, dim=-1)
  probs = probability[0].detach().cpu().numpy()
  confidences = {label: float(probs[i]) for i, label in enumerate(labels)} 
  return confidences

* **Build a Gradio web demo of your image classifier and `launch()` it**

Create a `gradio.Interface` and launch it! For image classification, the input component should be `"image"` and output should be a `"label"`

In [ ]:
import gradio as gr

interface = # FILL HERE

interface.launch()

# Step 6: Trying your Model with "Real World" Data!

* **Use the share link created above to open up your app on your phone**

Now test your model on some real images of plants -- either images you find online or those outside your house (they don't have be bean plants for this part). What do you notice about the kinds of predictions your model makes?  

[ANSWER HERE]

# This project is from [Abubakar Abid's](https://www.linkedin.com/in/abid15/) course: *Building Computer Vision Applications* on CoRise. Learn more about the course [here](https://corise.com/course/computer-vision).

<img src="https://slack-imgs.com/?c=1&o1=ro&url=https%3A%2F%2Fcorise.com%2F_next%2Fstatic%2Fmedia%2Fcorise.638d5854.png" width="400px">
